In [1]:
!pip3 install pymongo

In [2]:
# Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
#30% of the company staff have at least 1 child.
#Developers like to be near successful tech startups that have raised at least 1 Million dollars.
#Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
#Account managers need to travel a lot.
#Everyone in the company is between 25 and 40, give them some place to go party.
#The CEO is vegan.
#If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
#The office dog—"Pepe" needs a hairdresser every month. Ensure there's one not too far away.

In [3]:
#importing necessary libraries for geo_visualization
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv
from pymongo import MongoClient

In [4]:
conn = MongoClient("localhost:27017")

In [5]:
# choose database to work with
db = conn.get_database("ironhack")

# choose collection to work with
collection = db.get_collection("companies")

In [6]:
collection.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [7]:
c = collection

In [8]:
data_category = c.find({"category_code": "games_video"})

In [9]:
category_gaming = list(data_category)

In [10]:
len(category_gaming)

1083

In [11]:
data_dublin = c.find({"offices.city": "Dublin"})

In [12]:
location_dublin = list(data_dublin)

In [13]:
len(location_dublin)

66

In [14]:
#collection.find_one({"offices.city": "Dublin"})

In [15]:
pd.DataFrame(c.find(
    {"$and": [
        {"offices.city": 'Dublin'},
        {"total_money_raised": {'$gt': '1.0'}}
    ]},
    {"name": 1, "rating": 1, "total_money_raised": 1, 'offices.latitude':1, 'offices.longitude':1}
))

,_id,name,total_money_raised,offices
0,52cdef7c4bab8bd6752986b7,eRepublik,€2.75M,"[{'latitude': 40.43094, 'longitude': -3.695289..."
1,52cdef7d4bab8bd675298963,Iceberg,€400k,"[{'latitude': 53.344104, 'longitude': -6.267494}]"
2,52cdef7d4bab8bd675298aec,Dial2Do,€1M,"[{'latitude': 53.34202, 'longitude': -6.250684}]"
3,52cdef7d4bab8bd675299e60,payByMobile,€750k,"[{'latitude': None, 'longitude': None}]"
4,52cdef7e4bab8bd67529b0e2,DediServe,€1M,"[{'latitude': None, 'longitude': None}, {'lati..."
5,52cdef7e4bab8bd67529b59e,Branded Payment Solutions,€100k,"[{'latitude': 53.344104, 'longitude': -6.26749..."
6,52cdef7e4bab8bd67529b913,Xiam,€1M,"[{'latitude': None, 'longitude': None}]"
7,52cdef7f4bab8bd67529c55b,Clavis Technology,€3.66M,"[{'latitude': None, 'longitude': None}, {'lati..."


In [16]:
#Dial2Do, eRepublik

In [17]:
#latitude and longitude can be found in the office heading too

In [18]:
load_dotenv()

True

In [19]:
donde = "Dublin"
def geocode(address):
    """
    Saca las coordenadas de una dirección que le des.
    """
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [20]:
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()
data

{'standard': {'addresst': {},
  'city': 'Dublin',
  'prov': 'IE',
  'countryname': 'Ireland',
  'postal': {},
  'confidence': '0.90'},
 'longt': '-6.25415',
 'alt': {'loc': {'longt': '-6.30149',
   'prov': 'IE',
   'city': 'Dublin',
   'countryname': 'Ireland',
   'postal': 'D11Y895',
   'region': {},
   'latt': '53.40560'}},
 'elevation': {},
 'latt': '53.32306'}

In [21]:
Dublin = geocode(donde)

In [22]:
#longitud - latitud
Dublin

{'success': False,
 'error': {'code': '006',
  'message': 'Request Throttled. Over Rate limit: up to 2 per sec. See geocode.xyz/pricing',
  'requests': '3'}}

In [23]:
dublin = {'type': 'Point', 'coordinates': [-6.25415, 53.32306]}

In [24]:
initial_lat = 53.32306
initial_long = -6.25415

In [25]:
map_dublin = folium.Map(location = [53.32306,-6.25415], zoom_start = 15)
#map_dublin

In [26]:
#Adding Dial2Do
dial2do = Marker(location=[initial_lat,initial_long], tooltip = "Dial2Do")
dial2do.add_to(map_dublin)

In [27]:
#Location Dial2do
dial2do_lat = 53.34202
dial2do_long = -6.250684

In [28]:
#creating the icon for Dial2do
icono = Icon(color = "blue",
             prefix = "fa",
             icon = "briefcase",
             icon_color = "black",
             tooltip = "Dial2do"
)
loc = [dial2do_lat, dial2do_long]
#saving the marker in one variable
marker_dial2do = Marker(location = loc, icon = icono)
#let's add the marker to the original dublin map where we already had Dial2do location saved
marker_dial2do.add_to(map_dublin)
#map_dublin

In [29]:
#Adding Iceberg (even though raising <€1.0M)
#Location Iceberg
iceberg_lat =  53.344104
iceberg_long = -6.267494

#creating the icon for Iceberg
icono = Icon(color = "green",
             prefix = "fa",
             icon = "briefcase",
             icon_color = "black",
             tooltip = "Iceberg"
)
loc = [iceberg_lat, iceberg_long]
#saving the marker in one variable
marker_iceberg = Marker(location = loc, icon = icono)
#let's add the marker to the original dublin map where we already had Dial2do location saved
marker_iceberg.add_to(map_dublin)

#map_dublin

In [30]:
#Adding Clavis Technologies (searched coordinates in google maps)
#Location Clavis
clavis_lat =  53.346976
clavis_long = -6.258292

#creating the icon for Clavis
icono = Icon(color = "red",
             prefix = "fa",
             icon = "briefcase",
             icon_color = "black",
             tooltip = "Clavis Tech"
)
loc = [clavis_lat, clavis_long]
#saving the marker in one variable
marker_clavis = Marker(location = loc, icon = icono)
#let's add the marker to the original dublin map where we already had Dial2do location saved
marker_clavis.add_to(map_dublin)

map_dublin

In [31]:
clavis_tech = {'type': 'Point', 'coordinates': [-6.258292, 53.346976]}

# Calling FourSquare API

### Starbucks

In [32]:
load_dotenv()

True

In [33]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [34]:
enlace = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [35]:
print(clavis_tech.get("coordinates")[0])

-6.258292


In [36]:
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": "starbucks",
          "limit" : 10
}

In [37]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [38]:
#data
data.keys()

dict_keys(['meta', 'response'])

In [39]:
decoding_data = data.get('response')

In [40]:
#for k,v in decoding_data.items(): print (k,v,"\n")

In [41]:
decoded = decoding_data.get('groups')[0]

In [42]:
#for k,v in decoded.items(): print(k,v,"\n")

In [43]:
starbucks = decoded.get('items')

In [44]:
starbucks[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '52f7a098498ed300d2fe9e6d',
  'name': 'Starbucks',
  'contact': {},
  'location': {'address': "56 O'Connell St",
   'lat': 53.34759138567366,
   'lng': -6.259555235697369,
   'labeledLatLngs': [{'label': 'display',
     'lat': 53.34759138567366,
     'lng': -6.259555235697369}],
   'distance': 108,
   'postalCode': '1',
   'cc': 'IE',
   'city': 'Dublin',
   'state': 'Dublin City',
   'country': 'Ireland',
   'formattedAddress': ["56 O'Connell St",
    'Dublin',
    'Dublin City',
    'Ireland']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount

In [45]:
from functools import reduce
import operator

In [46]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [47]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [48]:
new_list = []
for diccionario in starbucks:
    starbuckslist = {}
    starbuckslist["name"] = getFromDict(diccionario,mapa_nombre)
    starbuckslist["latitud"] = getFromDict(diccionario,latitud)
    starbuckslist["longitud"] = getFromDict(diccionario,longitud)
    new_list.append(starbuckslist)

In [49]:
dataframestarbucks = pd.DataFrame(new_list)
dataframestarbucks.head()

,name,latitud,longitud
0,Starbucks,53.347591,-6.259555
1,Starbucks,53.350007,-6.259033
2,Starbucks,53.349626,-6.261584
3,Starbucks,53.347545,-6.263520
4,Starbucks,53.346098,-6.259072


In [50]:
import geopandas as gpd

In [51]:
gdf = gpd.GeoDataFrame(dataframestarbucks, geometry=gpd.points_from_xy(dataframestarbucks.longitud, dataframestarbucks.latitud))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,Starbucks,53.347591,-6.259555,POINT (-6.25956 53.34759)
1,Starbucks,53.350007,-6.259033,POINT (-6.25903 53.35001)
2,Starbucks,53.349626,-6.261584,POINT (-6.26158 53.34963)
3,Starbucks,53.347545,-6.263520,POINT (-6.26352 53.34755)
4,Starbucks,53.346098,-6.259072,POINT (-6.25907 53.34610)


In [52]:
from cartoframes.viz import Map, Layer, popup_element

In [53]:
Map(Layer(gdf, popup_hover=[popup_element("name","Starbucks")]))

### Irish Pubs

In [54]:
#Looking for some place to party - we will try focusing on the River Liffey surroundings
enlace = 'https://api.foursquare.com/v2/venues/explore'
irish_pubs = "52e81612bcbc57f1066b7a06"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": "irish_pubs",
          "limit" : 10
}

In [55]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [56]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
irish_pubs = decoded.get('items')

In [57]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [58]:
new_list2 = []
for diccionario in irish_pubs:
    irishpubslist = {}
    irishpubslist["name"] = getFromDict(diccionario,mapa_nombre)
    irishpubslist["latitud"] = getFromDict(diccionario,latitud)
    irishpubslist["longitud"] = getFromDict(diccionario,longitud)
    new_list2.append(irishpubslist)
    
dataframeirish = pd.DataFrame(new_list2)
#dataframeirish.head()

In [59]:
gdf = gpd.GeoDataFrame(dataframeirish, geometry=gpd.points_from_xy(dataframeirish.longitud, dataframeirish.latitud))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,Irish Whiskey Museum,53.344338,-6.259548,POINT (-6.25955 53.34434)
1,The Quays,53.345554,-6.263121,POINT (-6.26312 53.34555)
2,Irish Rock ‘N’ Roll Museum Experience,53.344976,-6.264444,POINT (-6.26444 53.34498)
3,Irish Film Institute (IFI),53.344667,-6.265295,POINT (-6.26529 53.34467)
4,Irish Design Shop,53.342667,-6.263078,POINT (-6.26308 53.34267)


In [60]:
Map(Layer(gdf, popup_hover=[popup_element("name","Irish Pubs")]))

### Veterinarian

In [64]:
#Looking for dog vet service
enlace = 'https://api.foursquare.com/v2/venues/explore'
vet = "4d954af4a243a5684765b473"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": "vet",
          "limit" : 10
}

In [65]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [66]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
vet = decoded.get('items')

In [67]:
new_list3 = []
for diccionario in vet:
    vetlist = {}
    vetlist["name"] = getFromDict(diccionario,mapa_nombre)
    vetlist["latitud"] = getFromDict(diccionario,latitud)
    vetlist["longitud"] = getFromDict(diccionario,longitud)
    new_list3.append(vetlist)
    
dataframevet = pd.DataFrame(new_list3)
#dataframevet.head()

In [121]:
gdf = gpd.GeoDataFrame(dataframevet, geometry=gpd.points_from_xy(dataframevet.longitud, dataframevet.latitud))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,The Porterhouse,53.345159,-6.267517,POINT (-6.26752 53.34516)
1,Duleek Vet Hospital,53.655193,-6.416615,POINT (-6.41662 53.65519)
2,Vet Farm Supplies,53.525459,-7.344558,POINT (-7.34456 53.52546)
3,Shannon Vet Surgery,53.423706,-7.983576,POINT (-7.98358 53.42371)


In [122]:
Map(Layer(gdf, popup_hover=[popup_element("name","Vets")]))

In [100]:
#Looking for vegan rests
enlace = 'https://api.foursquare.com/v2/venues/explore'
vegano = "4bf58dd8d48988d1d3941735"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": "vegano",
          "limit" : 15
}

In [101]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)
data

{'meta': {'code': 200, 'requestId': '5fb39898a5d23e3a41729898'},
 'response': {'warning': {'text': 'There aren\'t a lot of results for "vegano." Try something more general, reset your filters, or expand the search area.'},
  'suggestedRadius': 1809,
  'headerLocation': 'South-East Inner City',
  'headerFullLocation': 'South-East Inner City, Dublin',
  'headerLocationGranularity': 'neighborhood',
  'query': 'vegano',
  'totalResults': 0,
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [102]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
restaurantes = decoded.get("items")

In [103]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [104]:
new_list4 = []
for diccionario in restaurantes:
    veglist = {}
    veglist["name"] = getFromDict(diccionario,mapa_nombre)
    veglist["latitud"] = getFromDict(diccionario,latitud)
    veglist["longitud"] = getFromDict(diccionario,longitud)
    new_list4.append(veglist)
    
dataframeveg = pd.DataFrame(new_list4)
dataframeveg.head()

""


In [105]:
#Looking for airport connections
enlace = 'https://api.foursquare.com/v2/venues/explore'
airportservice = '56aa371be4b08b9a8d57352f'
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": "airportservice",
          "limit" : 20
}

In [106]:
resp_tram = requests.get (url = enlace, params = parametros)
data_tram = json.loads(resp.text)
data_tram

{'meta': {'code': 200, 'requestId': '5fb39898a5d23e3a41729898'},
 'response': {'warning': {'text': 'There aren\'t a lot of results for "vegano." Try something more general, reset your filters, or expand the search area.'},
  'suggestedRadius': 1809,
  'headerLocation': 'South-East Inner City',
  'headerFullLocation': 'South-East Inner City, Dublin',
  'headerLocationGranularity': 'neighborhood',
  'query': 'vegano',
  'totalResults': 0,
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [107]:
decoding_data = data_tram.get('response')
decoded = decoding_data.get('groups')[0]
trams = decoded.get("items")

In [108]:
map_name = ["venue","name"]
lat = ["venue", "location","lat"]
long = ["venue","location","lng"]

In [109]:
new_list_tram = []
for diccionario in trams:
    airlist = {}
    airlist["name"] = getFromDict(diccionario,mapa_nombre)
    airlist["latitud"] = getFromDict(diccionario,latitud)
    airlist["longitud"] = getFromDict(diccionario,longitud)
    new_list_tram.append(airlist)
    
dataframe_trams = pd.DataFrame(new_list_tram)
dataframe_trams.head()

""


In [110]:
#Base map is the location of Clavis Tech
map_final = Map(location=[53.346976,-6.258292],zoom_start=15)

In [111]:
dataframestarbucks.head()

,name,latitud,longitud,geometry
0,Starbucks,53.347591,-6.259555,POINT (-6.25956 53.34759)
1,Starbucks,53.350007,-6.259033,POINT (-6.25903 53.35001)
2,Starbucks,53.349626,-6.261584,POINT (-6.26158 53.34963)
3,Starbucks,53.347545,-6.263520,POINT (-6.26352 53.34755)
4,Starbucks,53.346098,-6.259072,POINT (-6.25907 53.34610)


In [112]:
# Calculate Distance Between GPS Points

import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [113]:
#Let's see which starbucks is closer to Clavis Tech
clavis_tech_coord = 53.346976,  -6.258292
starbucks = {
    'Starbucks_0': (53.347591,  -6.259555),
    'Starbucks_1': (53.350007,  -6.259033),
    'Starbucks_2': (53.349626, -6.261584),
    'Starbucks_3': (53.347545,  -6.263520),
    'Starbucks_4': (53.346098,  -6.259072)
}

for places, coord in starbucks.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see the closest Starbucks is c.100m away -> chosen starbucks

Starbucks_0 108.22086756471793
Starbucks_1 340.69816585905284
Starbucks_2 366.951514430256
Starbucks_3 352.8512306094474
Starbucks_4 110.5404164190214


In [114]:
dataframeirish.head()

,name,latitud,longitud,geometry
0,Irish Whiskey Museum,53.344338,-6.259548,POINT (-6.25955 53.34434)
1,The Quays,53.345554,-6.263121,POINT (-6.26312 53.34555)
2,Irish Rock ‘N’ Roll Museum Experience,53.344976,-6.264444,POINT (-6.26444 53.34498)
3,Irish Film Institute (IFI),53.344667,-6.265295,POINT (-6.26529 53.34467)
4,Irish Design Shop,53.342667,-6.263078,POINT (-6.26308 53.34267)


In [115]:
#Let's now calculate distantes to irish pubs
clavis_tech_coord = 53.346976,  -6.258292
irish_pubs = {
    'Irish Whiskey Museum': (53.344338,  -6.259548),
    'The Quays': (53.345554,  -6.263121),
    'Irish Rock ‘N’ Roll Museum Experience': (53.344976, -6.264444),
    'Irish Film Institute (IFI)': (53.344667,  -6.265295),
    'Irish Design Shop': (53.342667,  -6.263078)
}

for places, coord in irish_pubs.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see there are many irish pubs nearby: Irish Whiskey Museum and the Quays less than 400m away

Irish Whiskey Museum 305.0374232836449
The Quays 357.5307752212451
Irish Rock ‘N’ Roll Museum Experience 465.13649321082136
Irish Film Institute (IFI) 531.2096372569788
Irish Design Shop 575.0654827285944


In [116]:
dataframevet.head()

,name,latitud,longitud,geometry
0,The Porterhouse,53.345159,-6.267517,POINT (-6.26752 53.34516)
1,Duleek Vet Hospital,53.655193,-6.416615,POINT (-6.41662 53.65519)
2,Vet Farm Supplies,53.525459,-7.344558,POINT (-7.34456 53.52546)
3,Shannon Vet Surgery,53.423706,-7.983576,POINT (-7.98358 53.42371)


In [120]:
#Let's now calculate distantes to dog vet services
clavis_tech_coord = 53.346976,  -6.258292
vet_serv = {
    'The Porterhouse': (53.345159,  -6.267517),
    'Duleek Vet Hospital': (53.655193,  -6.416615),
    'Vet Farm Supplies': (53.525459, -7.344558),
    'Shannon Vet Surgery': (53.423706,  -7.983576)
}

for places, coord in vet_serv.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see there is a veterinarian where we could find a hairdresser for the dog (650m away)

The Porterhouse 645.0180090697779
Duleek Vet Hospital 35846.2829981018
Vet Farm Supplies 74662.12139205004
Shannon Vet Surgery 114768.11073931814
